In [1]:
import pandas as pd
import numpy as np

In [3]:
stats = pd.read_csv('footywire_stats.csv')
games = pd.read_csv('footywire_games.csv')

In [71]:
stats

,Date,Season,Round,Venue,Player,Team,Opposition,Status,Match_id,GA,...,FA,AF,SC,CCL,SCL,SI,MG,TO,ITC,T5
0,2015-04-02,2015,1,MCG,Bryce Gibbs,Carlton,Richmond,Home,5964,0.0,...,2.0,96.0,82.0,2.0,2.0,8.0,466.0,6.0,3.0,0.0
1,2015-04-02,2015,1,MCG,Tom Bell,Carlton,Richmond,Home,5964,2.0,...,0.0,108.0,115.0,0.0,1.0,10.0,475.0,4.0,1.0,3.0
2,2015-04-02,2015,1,MCG,Sam Docherty,Carlton,Richmond,Home,5964,1.0,...,0.0,107.0,147.0,0.0,2.0,6.0,287.0,2.0,12.0,1.0
3,2015-04-02,2015,1,MCG,Chris Judd,Carlton,Richmond,Home,5964,4.0,...,2.0,93.0,108.0,4.0,2.0,8.0,474.0,5.0,2.0,1.0
4,2015-04-02,2015,1,MCG,Kade Simpson,Carlton,Richmond,Home,5964,0.0,...,0.0,97.0,103.0,0.0,0.0,4.0,269.0,4.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98866,2025-07-26,2025,20,Optus Stadium,Sandy Brock,West Coast,Fremantle,Away,11357,0.0,...,0.0,30.0,41.0,0.0,0.0,2.0,163.0,1.0,1.0,0.0
98867,2025-07-26,2025,20,Optus Stadium,Jobe Shanahan,West Coast,Fremantle,Away,11357,0.0,...,0.0,34.0,15.0,0.0,0.0,1.0,10.0,3.0,0.0,1.0
98868,2025-07-26,2025,20,Optus Stadium,Matt Owies,West Coast,Fremantle,Away,11357,0.0,...,0.0,16.0,25.0,0.0,0.0,0.0,7.0,0.0,1.0,1.0
98869,2025-07-26,2025,20,Optus Stadium,Tyrell Dewar,West Coast,Fremantle,Away,11357,0.0,...,0.0,15.0,10.0,0.0,0.0,1.0,50.0,0.0,0.0,0.0


In [4]:
games['AwayWin'] = games['Away.Points'] > games['Home.Points']
games['HomeWin'] = games['Away.Points'] < games['Home.Points']
games['Draw'] = games['Away.Points'] == games['Home.Points']
games[['AwayWin', 'HomeWin', 'Draw']] = games[['AwayWin', 'HomeWin', 'Draw']].astype(int)

games['Date'] = pd.to_datetime(games['Date'])
games['Season'] = games['Date'].dt.year

merge_stats = stats[['Match_id', 'Round', 'Team', 'Season']].rename(columns={'Team': 'Home.Team'})
games = games.merge(merge_stats, on=['Round', 'Home.Team', 'Season']).drop_duplicates().reset_index(drop=True)

finals_map = {
            'Qualifying Final': 25,
            'Elimination Final': 25,
            'Semi Final': 26,
            'Preliminary Final': 27,
            'Preliminary Finals': 27,
            'Grand Final': 28}
games['Round'] = games['Round'].replace(finals_map)
games['Round'] = games['Round'].replace(r'Round (\d+)', r'\1', regex=True)
games['Round'] = games['Round'].astype(int)

stats['Round'] = stats['Round'].replace(finals_map)
stats['Round'] = stats['Round'].replace(r'Round (\d+)', r'\1', regex=True)
stats['Round'] = stats['Round'].astype(int)

In [201]:
stats_copy = stats.copy()
#stats_copy['Missing_Players'] = [ ['Heath Chapman'] for _ in range(len(stats_copy))]
new_row = pd.DataFrame([{'Match_id': 999999, 'Team': 'Western Bulldogs', 'Venue': 'Marvel Stadium', 'Season': 2025, 'Round': 21, 'Date': '2025-07-31', 'Missing_Players': ['Caleb Poulter'], 'Returning_Players': ['Nick Daicos']}])
stats_copy = pd.concat([stats_copy, new_row])



In [217]:
def get_last_10_rounds(matchid, team, stats):
    team_stats = stats[(stats['Team'] == team) & 
    (~stats['Player'].isin(stats['Missing_Players'].iloc[-1])) | 
    (stats['Player'].isin(stats['Returning_Players'].iloc[-1]))].copy() 
     
    #Filters stats to have only the current team
    
    current = team_stats[team_stats['Match_id'] == matchid]['Round'].max() 
    #Defines current as the most recent round
    return team_stats
    if matchid == 999999:
        last_round_players = team_stats[team_stats['Round'] == current - 1]
        last_round_players['Round'] = current
        last_round_players['Season'] = team_stats['Season'].values[-1]
        last_round_players['Match_id'] = 999999
        team_stats = team_stats[team_stats['Match_id'] != 999999]
        team_stats = pd.concat([team_stats, last_round_players])
    
    current_season_rounds = []
    remaining_rounds = 0
    previous_season = None
    #Defines variables for later use
    
    current_year = team_stats.loc[team_stats['Match_id'] == matchid, 'Season'].values[0]
    years = [current_year]
    #Defines the current season
    
    for i in range(1, 11):
        if current-i > -1:
            current_season_rounds.append(current-i)
        else:
            previous_season = True
            remaining_rounds += 1
    #Adds the last 10 games from this season to a list
    
    if previous_season and current_year != 2015:
        previous_season_rounds = []
        previous_year = current_year - 1
        previous_season = team_stats[team_stats['Season'] == previous_year]
        last_round = previous_season['Round'].max()
    #Finds the last round the team played in the previous season 
        
        for i in range(0, remaining_rounds + 1):
            previous_season_rounds.append(last_round-i)
        years = [current_year, previous_year]
    #Adds the games from last season to a list if they are within the last 10 games

        team_stats = team_stats[
            ((team_stats['Season'] == current_year) & (team_stats['Round'].isin(current_season_rounds))
            )
            |
            ((team_stats['Season'] == previous_year) & (team_stats['Round'].isin(previous_season_rounds))
            )
        ]
        return team_stats
    #Returns the rounds played in the current and previous seasons

    else:
        team_stats = team_stats[
            ((team_stats['Season'] == current_year) & (team_stats['Round'].isin(current_season_rounds))
            )
        ]
    
        return team_stats
        
    #Returns the rounds played in the current season

In [218]:
rounds = get_last_10_rounds(999999, 'Western Bulldogs', stats_copy)

In [219]:
(rounds['Player']=='Nick Daicos').value_counts()

Player
False    5546
True       88
Name: count, dtype: int64

In [220]:
def find_player_off(matchid, team, stats):
    scores_stats = get_last_10_rounds(matchid, team, stats)
    #Defines the stats for the last 10 games
    
    scores_stats = scores_stats.groupby('Player', as_index=False)[[
        'K', 'M', 'HB', 'G', 'B', 'HO', 'I50', 'CL', 'FF', 'MI5', 
        'GA', 'SI', 'MG', 'ED', 'TOG']].mean()
    #Finds the averages for each player over each stat
    
    scores_stats['OffensiveRating'] = (
        2.0 * scores_stats['K'] +                    # Kicks: key ball movement
        1.0 * scores_stats['HB'] +                   # Handballs: secondary ball movement
        1.5 * scores_stats['M'] +                    # Marks: possession retention
        6.0 * scores_stats['G'] +                    # Goals: primary scoring
        1.0 * scores_stats['B'] +                    # Behinds: partial scoring
        0.5 * scores_stats['HO'] +                   # Hitouts: minor offensive contribution
        3.0 * scores_stats['I50'] +                  # Inside 50s: advancing play
        2.0 * scores_stats['CL'] +                   # Clearances: initiating play
        1.0 * scores_stats['FF'] +                   # Frees For: advantage gained
        4.0 * scores_stats['MI5'] +                  # Marks Inside 50: scoring opportunity
        3.0 * scores_stats['GA'] +                   # Goal Assists: direct scoring contribution
        2.5 * scores_stats['SI'] +                   # Score Involvements: broader scoring impact
        0.01 * scores_stats['MG'] +                  # Metres Gained: small weight for territory
        1.5 * scores_stats['ED']) * (scores_stats['TOG'] / 100.0)                # Normalize by % Time on Ground
    #Calculates the offensive rating based on the average stats for each player
    
    return scores_stats[['Player', 'OffensiveRating']]


In [222]:
off = find_player_off(999999, 'Western Bulldogs', stats_copy)

In [139]:
def find_player_def(matchid, team, stats):
    scores_stats = get_last_10_rounds(matchid, team, stats)
    #Defines the stats for the last 10 games

    scores_stats = scores_stats.groupby('Player', as_index=False)[[
        'K', 'HB', 'M', 'CP', 'CM', 'One.Percenters', 'T', 'R50', 
        'CCL', 'SCL', 'CG', 'FA', 'TOG', 'DE', 'ITC', 'TO']].mean()
    #Calculates the average over each stat for every player
    
    scores_stats['DefensiveRating'] = (
        4 * scores_stats['T'] +               # Tackles: Stops opponents.
        3 * scores_stats['One.Percenters'] +  # One-Percenters: Small defensive acts.
        2.5 * scores_stats['R50'] +           # Rebound 50s: Clears defensive zone.
        2.5 * scores_stats['CM'] +            # Contested Marks: Marks under pressure.
        2.5 * scores_stats['ITC'] +           # Intercepts: Steals opposition ball.
        1.5 * scores_stats['CP'] +            # Contested Possessions: Wins contested ball.
        1.5 * scores_stats['CCL'] +           # Centre Clearances: Wins centre bounces.
        1.0 * scores_stats['SCL'] +           # Stoppage Clearances: Wins stoppage ball.
        0.5 * scores_stats['M'] +             # Marks: General ball catches.
        0.3 * scores_stats['K'] +             # Kicks: Ball movement.
        0.3 * scores_stats['HB'] +            # Handballs: Ball passing.
        0.1 * scores_stats['DE'] -            # Disposal Efficiency: Accurate disposals.
        2.5 * scores_stats['TO'] -            # Turnovers: Loses possession.
        2 * scores_stats['CG'] -              # Clangers: Defensive mistakes.
        1.5 * scores_stats['FA']              # Free Kicks Against: Concedes free kicks.
    ) * (scores_stats['TOG'] / 100)           # Time on Ground (%): Adjusts for play time.

    return scores_stats[['Player', 'DefensiveRating']]

def find_team_scores(matchid, team, stats):
    # Square the individual ratings before taking the mean
    player_def_scores = find_player_def(matchid, team, stats)['DefensiveRating']
    player_off_scores = find_player_off(matchid, team, stats)['OffensiveRating']
    
    player_def = float(round((player_def_scores ** 2).mean(), 2))
    player_off = float(round((player_off_scores ** 2).mean(), 2))
    
    return [matchid, team, player_off, player_def]

from sklearn.preprocessing import MinMaxScaler
def add_team_and_net_ratings(games: pd.DataFrame, stats: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()

    # Compute team ratings once per game/team
    games_subset['AwayStats'] = games_subset.apply(
        lambda row: find_team_scores(row['Match_id'], row['Away.Team'], stats),
        axis=1
    )
    games_subset['HomeStats'] = games_subset.apply(
        lambda row: find_team_scores(row['Match_id'], row['Home.Team'], stats),
        axis=1
    )
    # Extract offensive and defensive ratings
    games_subset[['OffensiveRatingAway', 'DefensiveRatingAway']] = games_subset['AwayStats'].apply(lambda x: pd.Series(x[2:4]))
    games_subset[['OffensiveRatingHome', 'DefensiveRatingHome']] = games_subset['HomeStats'].apply(lambda x: pd.Series(x[2:4]))
    # Calculate team ratings
    games_subset['AwayTeamRating'] = games_subset['OffensiveRatingAway'] - games_subset['DefensiveRatingHome']
    games_subset['HomeTeamRating'] = games_subset['OffensiveRatingHome'] - games_subset['DefensiveRatingAway']
    # Calculate net rating (away - home)
    games_subset['AwayNetRating'] = games_subset['AwayTeamRating'] - games_subset['HomeTeamRating']
    games_subset['HomeNetRating'] = games_subset['HomeTeamRating'] - games_subset['AwayTeamRating']
    scaler = MinMaxScaler(feature_range=(0, 1))
    games_subset[['HomeNetRating', 'AwayNetRating']] = scaler.fit_transform(games_subset[['HomeNetRating', 'AwayNetRating']])
    return games_subset.drop(columns=['HomeStats', 'AwayStats'])

In [190]:
def get_last_10_games(matchid, team, games):
    team_games = games[(games['Home.Team'] == team) | (games['Away.Team'] == team)].copy()
    
    current = team_games[team_games['Match_id'] == matchid]['Round'].max()
    current_season_rounds = []
    remaining_rounds = 0
    previous_season = None
    current_year = team_games.loc[team_games['Match_id'] == matchid, 'Season'].values[0]
    for i in range(1, 11):
        if current-i > -1:
            current_season_rounds.append(current-i)
        else:
            previous_season = True
            remaining_rounds += 1
    if previous_season and current_year != 2012:
        previous_season_rounds = []
        previous_year = current_year - 1
        previous_season = team_games[team_games['Season'] == previous_year]
        last_round = previous_season['Round'].max()
        for i in range(0, remaining_rounds + 1):
            previous_season_rounds.append(last_round-i)

        team_games = team_games[((team_games['Season'] == current_year
                                     ) & (team_games['Round'].isin(current_season_rounds)
                                         ) | (team_games['Season'] == previous_year
                                             ) & (team_games['Round'].isin(previous_season_rounds)))]


        return team_games
    else:
        team_games = team_games[(team_games['Season'] == current_year
                                     ) & (team_games['Round'].isin(current_season_rounds)
                                         )]
        return team_games


In [151]:
get_last_10_games(11356, 'Adelaide', games)


,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,Draw,Season,Match_id
2124,2025-05-17,13:20,10,MCG,Collingwood,Adelaide,78,68,0,1,0,2025,11270
2137,2025-05-25,12:40,11,Adelaide Oval,Adelaide,West Coast,128,62,0,1,0,2025,11283
2144,2025-05-31,19:35,12,SCG,Sydney,Adelaide,41,131,1,0,0,2025,11290
2148,2025-06-06,19:10,13,Adelaide Oval,Adelaide,Brisbane,68,63,0,1,0,2025,11294
2156,2025-06-13,19:40,14,UTAS Stadium,Hawthorn,Adelaide,47,44,0,1,0,2025,11302
2175,2025-06-29,15:15,16,MCG,Richmond,Adelaide,54,122,1,0,0,2025,11316
2184,2025-07-06,14:45,17,Adelaide Oval,Adelaide,Melbourne,90,77,0,1,0,2025,11329
2187,2025-07-12,13:20,18,Marvel Stadium,Western Bulldogs,Adelaide,98,109,1,0,0,2025,11332
2202,2025-07-20,16:10,19,Adelaide Oval,Adelaide,Gold Coast,107,46,0,1,0,2025,11347


In [145]:
def form(matchid, team, games):
    games_10 = get_last_10_games(matchid, team, games)

    games_10['TeamWin'] = ((games_10['Away.Team'] == team) & (games_10['AwayWin'] == True)) | ((games_10['Home.Team'] == team) & (games_10['HomeWin'] == True))
    winrate = games_10['TeamWin'].mean()
    return winrate

In [147]:
games

,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,Draw,Season,Match_id
0,2015-04-02,19:20,1,MCG,Carlton,Richmond,78,105,1,0,0,2015,5964
1,2015-04-04,13:40,1,MCG,Melbourne,Gold Coast,115,89,0,1,0,2015,5965
2,2015-04-04,16:35,1,Accor Stadium,Sydney,Essendon,72,60,0,1,0,2015,5966
3,2015-04-04,18:20,1,Gabba,Brisbane,Collingwood,74,86,1,0,0,2015,5967
4,2015-04-04,19:20,1,Marvel Stadium,Western Bulldogs,West Coast,97,87,0,1,0,2015,5968
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2205,2025-07-25,19:50,20,ENGIE Stadium,GWS,Sydney,102,58,0,1,0,2025,11349
2206,2025-07-26,13:20,20,People First Stadium,Gold Coast,Brisbane,130,64,0,1,0,2025,11355
2207,2025-07-26,14:15,20,Optus Stadium,Fremantle,West Coast,126,77,0,1,0,2025,11357
2208,2025-07-26,19:35,20,Marvel Stadium,North Melbourne,Geelong,49,150,1,0,0,2025,11350


In [6]:


def add_winrate_columns(games: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()


    # Calculate form (win rate in last 10 games)
    games_subset['AwayForms'] = games_subset.apply(
        lambda row: form(row['Match_id'], row['Away.Team'], games),
        axis=1
    )
    
    games_subset['HomeForms'] = games_subset.apply(
        lambda row: form(row['Match_id'], row['Home.Team'], games),
        axis=1
    )

    # Net form (away - home, and vice versa)
    games_subset['AwayNetForm'] = games_subset['AwayForms'] - games_subset['HomeForms']
    games_subset['HomeNetForm'] = games_subset['HomeForms'] - games_subset['AwayForms']

    return games_subset.drop(columns = ['AwayForms', 'HomeForms'])



In [146]:
get_last_10_games(999999, 'Western Bulldogs', games_copy)

[[20, 19, 18, 17, 16, 15, 14, 13, 12, 11],
 None,
 [np.int64(2025)],
                      Date   Time  Round            Venue         Home.Team  \
 4     2015-04-04 00:00:00  19:20      1   Marvel Stadium  Western Bulldogs   
 10    2015-04-11 00:00:00  13:45      2              MCG          Richmond   
 24    2015-04-19 00:00:00  13:10      3     UTAS Stadium          Hawthorn   
 35    2015-04-26 00:00:00  16:40      4   Marvel Stadium  Western Bulldogs   
 38    2015-05-02 00:00:00  14:10      5              SCG            Sydney   
 ...                   ...    ...    ...              ...               ...   
 2176  2025-07-03 00:00:00  19:30     17   Marvel Stadium   North Melbourne   
 2187  2025-07-12 00:00:00  13:20     18   Marvel Stadium  Western Bulldogs   
 2195  2025-07-18 00:00:00  19:40     19            Gabba          Brisbane   
 2204  2025-07-25 00:00:00  19:20     20   Marvel Stadium          Essendon   
 0              2025-07-31    NaN     21   Marvel Stadium  Wes

In [183]:
games_copy = games.copy()
#stats_copy['Missing_Players'] = [ ['Heath Chapman'] for _ in range(len(stats_copy))]
new_row = pd.DataFrame([{'Match_id': 999999, 'Home.Team': 'Western Bulldogs', 'Away.Team': 'GWS', 'Venue': ' Marvel Stadium', 'Season': 2025, 'Round': 21, 'Date': '2025-07-31'}])
games_copy = pd.concat([games_copy, new_row])
games_copy

,Date,Time,Round,Venue,Home.Team,Away.Team,Home.Points,Away.Points,AwayWin,HomeWin,Draw,Season,Match_id
0,2015-04-02 00:00:00,19:20,1,MCG,Carlton,Richmond,78.0,105.0,1.0,0.0,0.0,2015,5964
1,2015-04-04 00:00:00,13:40,1,MCG,Melbourne,Gold Coast,115.0,89.0,0.0,1.0,0.0,2015,5965
2,2015-04-04 00:00:00,16:35,1,Accor Stadium,Sydney,Essendon,72.0,60.0,0.0,1.0,0.0,2015,5966
3,2015-04-04 00:00:00,18:20,1,Gabba,Brisbane,Collingwood,74.0,86.0,1.0,0.0,0.0,2015,5967
4,2015-04-04 00:00:00,19:20,1,Marvel Stadium,Western Bulldogs,West Coast,97.0,87.0,0.0,1.0,0.0,2015,5968
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2206,2025-07-26 00:00:00,13:20,20,People First Stadium,Gold Coast,Brisbane,130.0,64.0,0.0,1.0,0.0,2025,11355
2207,2025-07-26 00:00:00,14:15,20,Optus Stadium,Fremantle,West Coast,126.0,77.0,0.0,1.0,0.0,2025,11357
2208,2025-07-26 00:00:00,19:35,20,Marvel Stadium,North Melbourne,Geelong,49.0,150.0,1.0,0.0,0.0,2025,11350
2209,2025-07-26 00:00:00,19:40,20,Adelaide Oval,Adelaide,Port Adelaide,133.0,35.0,0.0,1.0,0.0,2025,11356


In [ ]:
get_last_10_games

In [7]:
def get_last_5_matchups(matchid, team1, team2, games):
    team_games = games[((games['Home.Team'] == team1) & (games['Away.Team'] == team2)
                       ) | ((games['Home.Team'] == team2) & (games['Away.Team'] == team1))].copy()
    
    team_games = team_games.tail(6).iloc[:-1]
    
    team_games['Team1Win'] = ((team_games['Away.Team'] == team1) & (team_games['AwayWin'] == 1)
                             ) | ((team_games['Home.Team'] == team1) & (team_games['HomeWin'] == 1))
    team_games['Team2Win'] = ((team_games['Away.Team'] == team2) & (team_games['AwayWin'] == 1)
                             ) | ((team_games['Home.Team'] == team2) & (team_games['HomeWin'] == 1))
    team1winrate = float(team_games['Team1Win'].astype(int).mean())
    team2winrate = float(team_games['Team2Win'].astype(int).mean())
    return [team1winrate, team2winrate]

def add_matchup_columns(games: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()

    # Apply get_last_5_matchups to compute win rates for each row
    matchup_winrates = games_subset.apply(
        lambda row: pd.Series(
            get_last_5_matchups(row['Match_id'], row['Home.Team'], row['Away.Team'], games)
        ),
        axis=1
    )

    # Assign column names to the result
    matchup_winrates.columns = ['HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate']

    # Add the new columns to the original subset
    games_subset[['HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate']] = matchup_winrates

    # Return the full DataFrame with added columns
    return games_subset

In [187]:
def get_last_5_venue(matchid, home_team, away_team, games):
    # Get the venue of the current game
    venue = games.loc[games['Match_id'] == matchid, 'Venue'].values[0]

    # Filter all games involving either team at that venue
    venue_games = games[((games['Home.Team'] == home_team) | (games['Away.Team'] == home_team) |
                         (games['Home.Team'] == away_team) | (games['Away.Team'] == away_team)) &
                        (games['Venue'] == venue)].copy()
    
    venue_games = venue_games.sort_values(['Season', 'Round'])
    # Get last 5 games at venue for home team (excluding current)
    home_venue_games = venue_games[((venue_games['Home.Team'] == home_team) | (venue_games['Away.Team'] == home_team)) &
                                   (venue_games['Match_id'] != matchid)].tail(5)
    # Get last 5 games at venue for away team (excluding current)
    away_venue_games = venue_games[((venue_games['Home.Team'] == away_team) | (venue_games['Away.Team'] == away_team)) &
                                   (venue_games['Match_id'] != matchid)].tail(5)
    # Calculate win rates
    home_wins = ((home_venue_games['Home.Team'] == home_team) & (home_venue_games['HomeWin'] == 1)) | \
                ((home_venue_games['Away.Team'] == home_team) & (home_venue_games['AwayWin'] == 1))
    away_wins = ((away_venue_games['Home.Team'] == away_team) & (away_venue_games['HomeWin'] == 1)) | \
                ((away_venue_games['Away.Team'] == away_team) & (away_venue_games['AwayWin'] == 1))
    home_winrate = home_wins.astype(int).mean() if not home_venue_games.empty else 0.0
    away_winrate = away_wins.astype(int).mean() if not away_venue_games.empty else 0.0
    return [home_winrate, away_winrate]

In [188]:
get_last_5_venue(999999, 'Western Bulldogs', 'GWS', games_copy)

[np.float64(0.8), np.float64(0.6)]

In [155]:
def add_venue_columns(games: pd.DataFrame, years) -> pd.DataFrame:
    games_subset = games[games['Season'].isin(years)].copy()

    # Apply get_last_5_venue to compute venue win rates for each row
    venue_winrates = games_subset.apply(
        lambda row: pd.Series(
            get_last_5_venue(row['Match_id'], row['Home.Team'], row['Away.Team'], games)
        ),
        axis=1
    )

    # Assign column names
    venue_winrates.columns = ['HomeVenueWinRate', 'AwayVenueWinRate']

    # Add the new columns to the subset
    games_subset[['HomeVenueWinRate', 'AwayVenueWinRate']] = venue_winrates

    return games_subset


In [65]:
def days_since_last_game(match_id, team, games):
    """
    Calculate days since the last game for a given team and match ID.
    
    Args:
        match_id: The ID of the match to evaluate.
        team: The team name to filter games for.
        games: DataFrame with columns ['Date', 'Home.Team', 'Away.Team', 'Match_id'].
    
    Returns:
        int: Number of days since the team's last game, or 0 if first game.
        None: If team or match_id is not found.
    """
    # Ensure Date is in datetime format
    games = games.copy()
    games['Date'] = pd.to_datetime(games['Date'])
    
    # Filter games where team is either Home or Away, sorted by date
    team_games = games[
        (games['Home.Team'] == team) | (games['Away.Team'] == team)
    ].copy().sort_values('Date').reset_index(drop=True)
    
    # Check if any games were found for the team
    if team_games.empty:
        return None
    
    # Find the current game
    current_game = team_games[team_games['Match_id'] == match_id]
    if current_game.empty:
        return None
    
    # Get the index of the current game
    current_index = current_game.index[0]
    
    # If it's the first game, return 0
    if current_index == 0:
        return 0
    
    # Get the previous game
    previous_game = team_games.iloc[current_index - 1]
    
    # Calculate days between current and previous game
    days_diff = (current_game['Date'].iloc[0] - previous_game['Date']).days
    if days_diff > 30:
        days_diff = 30
    return int(days_diff)

def add_days_columns(games, years):
    """
    Add columns for days since last game for home and away teams in a subset of games.
    
    Args:
        games: DataFrame with game data.
        years: List of seasons to filter (e.g., [2021]).
    
    Returns:
        DataFrame: Subset of games with new columns 'home_days_since' and 'away_days_since'.
    """
    # Filter games for the specified years
    games_subset = games[games['Season'].isin(years)].copy()
    
    # Check if subset is empty
    if games_subset.empty:
        return games_subset
    
    # Calculate days since last game for home and away teams
    games_subset['home_days_since'] = games_subset.apply(
        lambda row: days_since_last_game(row['Match_id'], row['Home.Team'], games),
        axis=1
    )
    games_subset['away_days_since'] = games_subset.apply(
        lambda row: days_since_last_game(row['Match_id'], row['Away.Team'], games),
        axis=1
    )
    games_subset['away_days_since_net'] = games_subset['away_days_since'] - games_subset['home_days_since']
    games_subset['home_days_since_net'] = games_subset['home_days_since'] - games_subset['away_days_since']

    games_subset = games_subset.drop(columns=['home_days_since', 'away_days_since'])

    scaler = MinMaxScaler(feature_range=(0, 1))
    
 
    games_subset[['away_days_since_net', 'home_days_since_net']] = scaler.fit_transform(games_subset[['away_days_since_net', 'home_days_since_net']])



    
    return games_subset


In [66]:
from sklearn.preprocessing import MinMaxScaler

def add_elo_ratings(games: pd.DataFrame, base_elo: int = 1500, k: int = 40) -> pd.DataFrame:
    # Sort by time
    df = games.sort_values(by=['Season', 'Date', 'Time']).copy()

    # Elo dictionary
    elo_ratings = {}
    home_elos, away_elos = [], []

    for _, row in df.iterrows():
        home, away = row['Home.Team'], row['Away.Team']
        margin = abs(row['Home.Points'] - row['Away.Points'])

        # Get ratings
        home_elo = elo_ratings.get(home, base_elo)
        away_elo = elo_ratings.get(away, base_elo)

        # Expected outcome
        expected_home = 1 / (1 + 10 ** ((away_elo - home_elo) / 400))
        expected_away = 1 - expected_home

        # Actual result
        home_score = 1 if row['HomeWin'] == 1 else 0
        away_score = 1 if row['AwayWin'] == 1 else 0

        # Margin of Victory Multiplier
        mov_mult = np.log(margin + 1) * (2.2 / ((home_elo - away_elo) * 0.001 + 2.2))

        # Update Elo ratings
        elo_ratings[home] = home_elo + k * mov_mult * (home_score - expected_home)
        elo_ratings[away] = away_elo + k * mov_mult * (away_score - expected_away)

        # Save pre-game Elo
        home_elos.append(home_elo)
        away_elos.append(away_elo)

    # Add Elo values to df
    df['HomeElo'] = home_elos
    df['AwayElo'] = away_elos
    df['NetHomeElo'] = df['HomeElo'] - df['AwayElo']
    df['NetAwayElo'] = -df['NetHomeElo']

    # Normalize difference features only (not absolute ratings)
    scaler = MinMaxScaler(feature_range=(0, 1))
    df[['NetHomeElo', 'NetAwayElo']] = scaler.fit_transform(df[['NetHomeElo', 'NetAwayElo']])

    return df[['Match_id', 'HomeElo', 'AwayElo', 'NetAwayElo', 'NetHomeElo']]



In [9]:
def add_all_features(games: pd.DataFrame, stats: pd.DataFrame, years: list) -> pd.DataFrame:
    # Compute all features on full history
    ratings_df = add_team_and_net_ratings(games, stats, years)
    forms_df = add_winrate_columns(games, years)
    matchups_df = add_matchup_columns(games, years)
    venues_df = add_venue_columns(games, years)
    days_df = add_days_columns(games, years)
    elo_df = add_elo_ratings(games) 
    # Filter the final set of rows by year
    filtered_games = games[games['Season'].isin(years)].copy()
    # Merge only the needed columns from feature sets
    filtered_games = filtered_games.merge(
        ratings_df[['Match_id', 'HomeNetRating', 'AwayNetRating']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        forms_df[['Match_id', 'HomeNetForm', 'AwayNetForm']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        matchups_df[['Match_id', 'HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        venues_df[['Match_id', 'HomeVenueWinRate', 'AwayVenueWinRate']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        days_df[['Match_id', 'home_days_since_net', 'away_days_since_net']],
        on='Match_id', how='left'
    )
    filtered_games = filtered_games.merge(
        elo_df[['Match_id', 'HomeElo', 'AwayElo', 'NetHomeElo', 'NetAwayElo']],
        on='Match_id', how='left'
    )
    
    # Select final columns
    final = filtered_games[[
        'Match_id', 'Season', 'Home.Team', 'Away.Team',
        'Home.Points', 'Away.Points',
        'HomeNetRating', 'AwayNetRating',
        'HomeNetForm', 'AwayNetForm',
        'HomeLast5MatchupWinRate', 'AwayLast5MatchupWinRate',
        'HomeVenueWinRate', 'AwayVenueWinRate',
        'home_days_since_net', 'away_days_since_net', 
        'NetHomeElo', 'NetAwayElo',
        'HomeWin', 'AwayWin'
    ]]
    return final

In [198]:
def predict_match(home_team, away_team, venue, season, round_n, date, hmissing_players, hreturning_players, amissing_players, areturning_players, model, feature_data, stats, X_train_columns):
    match_dict = {
        "Date": date,
        "Match_id": 999999, 
        "Season": season,
        "Home.Team": home_team,
        "Away.Team": away_team,
        "Venue": venue,
        "Round": round_n,
    }
    match_df = pd.DataFrame([match_dict])
    new_games = pd.concat([games, match_df], ignore_index=True)
    stats_df1 = pd.DataFrame([{"Match_id": 999999,
        "Season": season,
        "Team": home_team,
        "Venue": venue,
        "Round": round_n,
        "Missing_Players": hmissing_players,
        "Returning_Players": hreturning_players}])
                              
    stats_df2 = pd.DataFrame([{"Match_id": 999999,  
        "Season": season,
        "Team": away_team,
        "Venue": venue,
        "Round": round_n,
        "Missing_Players": amissing_players,
        "Returning_Players": areturning_players}])

                              
    new_stats = pd.concat([stats, stats_df1, stats_df2])

    match_features = add_all_features(
        new_games,
        stats=new_stats,  
        years=[season]
    ).query("Match_id == 999999")

    # Select features same as training
    X_new = match_features[X_train_columns]

    # Predict
    predicted_class = model.predict(X_new)[0]
    predicted_proba = model.predict_proba(X_new)[0]

    # Build output DataFrame
    result_df = match_features.copy()
    result_df["PredictedWinner"] = "Home" if predicted_class == 0 else "Away"
    result_df["PredictedProb_HomeWin"] = predicted_proba[0]
    result_df["PredictedProb_AwayWin"] = predicted_proba[1]

    return result_df


In [227]:

# List of Round 21, 2025 games
round21_games_2025 = [
    ['Western Bulldogs', 'GWS', 'Marvel Stadium', 2025, 21, '31-07-2025'],
    ['Adelaide', 'Hawthorn', 'Adelaide Oval', 2025, 21, '01-08-2025'],
    ['Melbourne', 'West Coast', 'Marvel Stadium', 2025, 21, '01-08-2025'],
    ['Gold Coast', 'Richmond', 'People First Stadium', 2025, 21, '02-08-2025'],
    ['Sydney', 'Essendon', 'SCG', 2025, 21, '02-08-2025'],
    ['Collingwood', 'Brisbane', 'MCG', 2025, 21, '02-08-2025'],
    ['St Kilda', 'North Melbourne', 'Marvel Stadium', 2025, 21, '02-08-2025'],
    ['Geelong', 'Port Adelaide', 'GMHBA Stadium', 2025, 21, '03-08-2025'],
    ['Fremantle', 'Carlton', 'Optus Stadium', 2025, 21, '03-08-2025']
]

# Assuming hmissing_players and amissing_players are provided or default to empty lists
# Example: Use ['Heath Chapman'] for Western Bulldogs as per your earlier query
missing_players_dict = {
    'Western Bulldogs': ['Heath Chapman'],
    'GWS': [],
    'Adelaide': [],
    'Hawthorn': [],
    'Melbourne': [],
    'West Coast': [],
    'Gold Coast': [],
    'Richmond': [],
    'Sydney': [],
    'Essendon': [],
    'Collingwood': [],
    'Brisbane': [],
    'St Kilda': [],
    'North Melbourne': [],
    'Geelong': [],
    'Port Adelaide': [],
    'Fremantle': [],
    'Carlton': []
}

returning_players_dict = {
    'Western Bulldogs': [],
    'GWS': [],
    'Adelaide': [],
    'Hawthorn': [],
    'Melbourne': [],
    'West Coast': [],
    'Gold Coast': [],
    'Richmond': [],
    'Sydney': [],
    'Essendon': [],
    'Collingwood': [],
    'Brisbane': [],
    'St Kilda': [],
    'North Melbourne': [],
    'Geelong': [],
    'Port Adelaide': [],
    'Fremantle': [],
    'Carlton': []
}

all_predictions = []
for game in round21_games_2025:
    home_team, away_team, venue, season, round_n, date = game
    # Normalize venue name
    venue = venue.strip().lower()
    # Get missing players (default to empty list if not specified)
    hmissing_players = missing_players_dict.get(home_team, [])
    amissing_players = missing_players_dict.get(away_team, [])
    hreturning_players = returning_players_dict.get(home_team, [])
    areturning_players = returning_players_dict.get(away_team, [])
    
    result = predict_match(
        home_team=home_team,
        away_team=away_team,
        venue=venue,
        season=season,
        round_n=round_n,
        date=date,
        hmissing_players=hmissing_players,
        amissing_players=amissing_players,
        hreturning_players=hreturning_players,
        areturning_players=areturning_players,
        model=best_model,
        feature_data=games,
        stats=stats,
        X_train_columns=X_train.columns
    )
    all_predictions.append(result)

final_predictions_df = pd.concat(all_predictions, ignore_index=True)
print(final_predictions_df)

   Match_id  Season         Home.Team        Away.Team  Home.Points  \
0    999999    2025  Western Bulldogs              GWS          NaN   
1    999999    2025          Adelaide         Hawthorn          NaN   
2    999999    2025         Melbourne       West Coast          NaN   
3    999999    2025        Gold Coast         Richmond          NaN   
4    999999    2025            Sydney         Essendon          NaN   
5    999999    2025       Collingwood         Brisbane          NaN   
6    999999    2025          St Kilda  North Melbourne          NaN   
7    999999    2025           Geelong    Port Adelaide          NaN   
8    999999    2025         Fremantle          Carlton          NaN   

   Away.Points  HomeNetRating  AwayNetRating  HomeNetForm  AwayNetForm  ...  \
0          NaN       0.647478       0.352522    -0.333333     0.333333  ...   
1          NaN       0.610540       0.389460     0.222222    -0.222222  ...   
2          NaN       0.687779       0.312221     0.2

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score

data = pd.read_csv('footywire_data.csv')
data = data[~data['Season'].isin([2015,2016,2017,2018,2019,2020,2021])]


X = data[['AwayNetForm', 'AwayNetRating', 'AwayLast5MatchupWinRate', 'AwayVenueWinRate', 'away_days_since_net', 'NetAwayElo']]
y = data['AwayWin']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Best parameters from tuning
best_params = {
    'max_depth': 10,
    'max_features': 'sqrt',
    'min_samples_leaf': 4,
    'min_samples_split': 10,
    'n_estimators': 300,
    'random_state': 42  # for reproducibility
}

best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Tuned Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Tuned Model Accuracy: 0.7407407407407407
              precision    recall  f1-score   support

           0       0.75      0.83      0.79        95
           1       0.72      0.61      0.66        67

    accuracy                           0.74       162
   macro avg       0.74      0.72      0.73       162
weighted avg       0.74      0.74      0.74       162



In [ ]:
import joblib
import os

# Define the path to save the model in the models/ folder
model_path = os.path.join('..', 'models', 'afl_random_forest_model.pkl')
joblib.dump(best_model, model_path)
print(f"Model saved to {model_path}")

In [228]:
final_prediction = final_predictions_df[["Home.Team", "Away.Team", "PredictedWinner",
    "PredictedProb_HomeWin", "PredictedProb_AwayWin"]]
final_prediction['PredictedProb_HomeWin'] = final_prediction['PredictedProb_HomeWin'].round(2)
final_prediction['PredictedProb_AwayWin'] = final_prediction['PredictedProb_AwayWin'].round(2)
final_prediction.rename(columns={'PredictedProb_HomeWin': 'Probability_of_Home_Win', 'PredictedProb_AwayWin': 'Probability_of_Away_Win'})


C:\Users\maxma\AppData\Local\Temp\ipykernel_75120\192916190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_prediction['PredictedProb_HomeWin'] = final_prediction['PredictedProb_HomeWin'].round(2)
C:\Users\maxma\AppData\Local\Temp\ipykernel_75120\192916190.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_prediction['PredictedProb_AwayWin'] = final_prediction['PredictedProb_AwayWin'].round(2)


,Home.Team,Away.Team,PredictedWinner,Probability_of_Home_Win,Probability_of_Away_Win
0,Western Bulldogs,GWS,Home,0.78,0.22
1,Adelaide,Hawthorn,Home,0.55,0.45
2,Melbourne,West Coast,Home,0.89,0.11
3,Gold Coast,Richmond,Home,0.87,0.13
4,Sydney,Essendon,Home,0.66,0.34
5,Collingwood,Brisbane,Home,0.89,0.11
6,St Kilda,North Melbourne,Home,0.84,0.16
7,Geelong,Port Adelaide,Home,0.78,0.22
8,Fremantle,Carlton,Home,0.74,0.26


In [ ]:
Home.Team	Away.Team	PredictedWinner	Probability_of_Home_Win	Probability_of_Away_Win
0	Western Bulldogs	GWS	Home	0.71	0.29
1	Adelaide	Hawthorn	Away	0.46	0.54
2	Melbourne	West Coast	Home	0.89	0.11
3	Gold Coast	Richmond	Home	0.90	0.10
4	Sydney	Essendon	Home	0.68	0.32
5	Collingwood	Brisbane	Home	0.84	0.16
6	St Kilda	North Melbourne	Home	0.83	0.17
7	Geelong	Port Adelaide	Home	0.66	0.34
8	Fremantle	Carlton	Home	0.67	0.33
